<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Разделение-исходных-данных" data-toc-modified-id="Разделение-исходных-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение исходных данных</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Проверка-качества-модели" data-toc-modified-id="Проверка-качества-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка качества модели</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Рекомендация тарифов оператора мобильной связи

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

**Цель исследования**: построить модель для задачи классификации, которая выберет подходящий для клиента тариф.

**Ход исследования**

Для построения модели предоставлены данные о поведении пользователей, которые уже перешли на новые тарифы. Предобработка данных уже проведена.

Построение модели пройдёт в четыре этапа:
 1. Обзор данных.
 2. Разделение исходных данных на обучающую, валидационную и тестовую выборки.
 3. Исследование качество разных моделей с разными гиперпараметрами.
 4. Проверка качества модели на тестовой выборке.

## Обзор данных

Для начала импортируем необходимые для работы библиотеки и модули

In [1]:
# импорт необходимых библиотек и модулей
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier

 Прочитаем файл с данными и сохраним в переменной.

In [2]:
# чтение файла с данными
df = pd.read_csv('datasets/users_behavior.csv')

Выведем на экран первые десять строк таблицы.

In [3]:
df.head(10) # получение первых 10 строк таблицы

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Изучим общую информацию о таблице.

In [4]:
df.info() # получение общей информации о данных в таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В таблице представлена информация о поведении одного пользователя за месяц. 

Согласно документации к данным:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Количество звонков и сообщений не может быть дробным, поэтому стоит привести тип данных в `сalls` и `messages` к целочисленному типу.

In [5]:
# приведение к целочисленному типу
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')

**Выводы**

Всего имеем 3214 объектов, признаки - количество СМС, звонков, суммарная длительность звонков в минутах и израсходованный интернет-трафик. Целевой признак - тариф, которым пользовался клиент. Данные готовы к работе. На следующем шаге разделим их на выборки.

## Разделение исходных данных 

Для машинного обучения необходимо, чтобы признаки были сохранены в отдельных переменных. Сохраним признаки в переменной `features`, а целевой признак - в переменной `target`.

In [6]:
# создадим переменные features и target
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Разобьем исходный набор данных на три выборки: обучающую, валидационную и тестовую в пропорции 60:20:20. Для этого воспользуемся функцией `train_test_split` в 2 этапа: сначала выделим обучающую выборку, а затем разделим оставшийся набор на валидационную и тестовую выборки.

In [7]:
# выделение обучающей выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)

# выделение валидационной и тестовой выборок
features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=0.5, random_state=12345, stratify=target_test)

# проверка размеров выборок
print('Количество объектов обучающей выборки:', features_train.shape[0])
print('Количество объектов валидационной выборки:', features_valid.shape[0])
print('Количество объектов тестовой выборки:', features_test.shape[0])

Количество объектов обучающей выборки: 1928
Количество объектов валидационной выборки: 643
Количество объектов тестовой выборки: 643


**Выводы**

Признаки и целевой признак были сохранены в отдельные переменные. Исходные данные разделены на обучающую (1928 объектов), валидационную и тестовую (по 643 объекта) выборки. Можно переходить к обучению моделей.

## Исследование моделей

Поскольку целевой признак (тариф) является категориальным и при этом имеет всего 2 категории, будем решать задачу бинарной классификации.

Рассмотрим три разных модели: 
* дерево решений,
* случайный лес,
* логистическую регрессию.

Каждую модель обучим на обучающей выборке и проверим на валидационной, а также сравним их качество с помощью метрики `accuracy`.

### Дерево решений

Построим модель решающего дерева. Для того, чтобы получить максимальное высокое качество предсказания, переберем в цикле разные варианты глубины дерева - от 1 до 10.

In [8]:
# построение модели дерева решений с подбором оптимальной глубины дерева
best_accuracy_tree = 0
best_max_depth = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_tree = model_tree.predict(features_valid)
    accuracy_tree = accuracy_score(target_valid, predictions_tree)
    if accuracy_tree > best_accuracy_tree:
        best_accuracy_tree = accuracy_tree.round(4)
        best_max_depth = depth
    print('max_depth =', depth, ':', accuracy_tree.round(4))
print()
print('Accuracy лучшей модели:', best_accuracy_tree,', максимальная глубина:', best_max_depth)

max_depth = 1 : 0.7403
max_depth = 2 : 0.7729
max_depth = 3 : 0.7776
max_depth = 4 : 0.7543
max_depth = 5 : 0.7854
max_depth = 6 : 0.7745
max_depth = 7 : 0.7869
max_depth = 8 : 0.8025
max_depth = 9 : 0.7823
max_depth = 10 : 0.7729

Accuracy лучшей модели: 0.8025 , максимальная глубина: 8


Наилучшие результаты продемонстрировала модель с максимальной глубиной, равной 8. Она дает 80,25% правильных ответов.

### Случайный лес

Перейдем к построению модели случайного леса. Подберем наилучший вариант гиперпараметров с помощью цикла.

In [9]:
# построение модели случайного леса с подбором количества оценщиков и оптимальной глубины дерева
best_accuracy_forest = 0
best_est = 0
best_max_depth = 0
for est in range(1, 11):
    for depth in range(1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model_forest.fit(features_train, target_train) 
        predictions_forest = model_forest.predict(features_valid)
        accuracy_forest = accuracy_score(target_valid, predictions_forest)
        if accuracy_forest > best_accuracy_forest:
            best_accuracy_forest = accuracy_forest.round(4)
            best_max_depth = depth
            best_est = est
        
print('Accuracy лучшей модели:', best_accuracy_forest, 
      ', количество оценщиков:', best_est, 
      ', максимальная глубина:', best_max_depth)

Accuracy лучшей модели: 0.8072 , количество оценщиков: 6 , максимальная глубина: 6


При 6 оценщиках с максимальной глубиной 6 получаем долю правильных ответов 0,8072. Таким образом, данная модель пока демонстрирует лучший результат, чем дерево решений.

### Логистическая регрессия

Теперь построим логистическую регрессию. 

In [10]:
# построение логистической регрессии
model_lr = LogisticRegression(random_state=12345, penalty='none')
model_lr.fit(features_train, target_train) 
predictions_lr = model_forest.predict(features_valid)
accuracy_lr = accuracy_score(target_valid, predictions_lr)
print('Accuracy модели:', accuracy_lr.round(4))

Accuracy модели: 0.7994


Модель логистической регрессии показала результат в 79,94% правильных ответов. 

**Выводы**

Самое высокое качество предсказания у модели случайного леса с количеством оценщиков и максимальной глубиной, равными 6 (accuracy = 0,8072). На втором месте дерево решений с accuracy = 0,8025. Самое низкое качество у логистической регрессии (0,7994). Выберем наилучшую модель - случайный лес.

## Проверка качества модели

Проверим качество выбранной модели на тестовой выборке.

In [11]:
# расчет accuracy на тестовой выборке
model = RandomForestClassifier(random_state=12345, n_estimators=6, max_depth=6)
model.fit(features_train, target_train) 
predictions = model_forest.predict(features_test)
predictions_accuracy = accuracy_score(target_test, predictions)
print('Accuracy тестовой выборки:',predictions_accuracy.round(4))

Accuracy тестовой выборки: 0.8025


На тестовой выборке получили 80,25% правильных ответов, что очень близко к результату на валидационной выборке. Таким образом, можно сказать, что модель обучена хорошо, переобучения нет.

## Общий вывод

В ходе работы были изучены данные о поведении клиентов, перешедших на тарифы "Смарт" и "Ультра" (количество СМС, звонков, суммарная длительность звонков в минутах и израсходованный интернет-трафик). 

Данные были разбиты на три выборки: обучающую (для обучения алгоритма), валидационную (для подбора гиперпараметров и выбора лучшей модели) и тестовую (для оценки качества модели).

Были построены следующие модели: дерево решений, случайный лес, логистическая регрессия, Для каждого из алгоритмов были подобраны гиперпараметры, дающие наибольшую долю правильных ответов (accuracy) в валидационной выборке. Самое высокое качество оказалось у модели случайного леса с количеством оценщиков и максимальной глубиной, равными 6. Доля правильных ответов составила 80,72%.

Проверка качества модели показала, что она обучена хорошо. Accuracy для валидационной и тестовых выборок практически совпадает. 

Таким образом, нам удалось построить систему, способную с 80-процентной точностью проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».